# Fix Tempest Sources to Work with MAC65VM03.09

In [11]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

Clone the historicalsources source dump.

In [57]:
!rm -rf tempest_modified
!git clone https://github.com/historicalsource/tempest tempest_modified

Cloning into 'tempest_modified'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 8), reused 45 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 174.01 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (8/8), done.


This patch removes the dependency (which our old version of MAC65 can't handle) on correctly assembling globals with an offset.

In [61]:
! cat 0001-Fix-byte-relocation-errors.patch

From 21308d40700723190fed6ed743d6cbdb134c1fde Mon Sep 17 00:00:00 2001
From: Robert Hogan <robhogan@gmail.com>
Date: Sat, 26 Oct 2024 14:35:28 +0100
Subject: [PATCH] Fix byte relocation errors

---
 ALDIS2.MAC |  29 +++++++++++++++--------------
 ALSCO2.MAC |  13 +++++++------
 ALTES2.MAC |  10 +++++-----
 ALVGUT.MAC |   2 +-
 ALVROM.MAC |  19 +++++++++++++++----
 ANVGAN.MAC | Bin 6144 -> 6226 bytes
 6 files changed, 43 insertions(+), 30 deletions(-)

diff --git a/ALDIS2.MAC b/ALDIS2.MAC
index f07c235..c6fdf5a 100644
--- a/ALDIS2.MAC
+++ b/ALDIS2.MAC
@@ -15,7 +15,7 @@








@@ -29,7 +29,8 @@









@@ -47,7 +48,7 @@ SPACG=0			;SUPPRESS SPACE GAME CODE








@@ -85,9 +86,9 @@ ZATVG2::











@@ -262,7 +263,7 @@ SBCSWI:	PHA








@@ -1114,9 +1115,9 @@ SHRSCA:			;CHANGE SHRAPNEL SCALE VARIABLE











@@ -1220,10 +1221,10 @@ CALMAG:












@@ -2335,9 +2336,9 @@ INIDSP:	JSR INITEM		;COPY SCORE TEMPLATE TO VECTOR RAM











@@ -3201,7 +3202,7 @@ WHITIP:	LDY VGY			;JU

Apply our patch that allows it to assemble and link correctly. 

In [60]:
!cd tempest_modified && git apply ../0001-Fix-byte-relocation-errors.patch 

../0001-Fix-byte-relocation-errors.patch:76: trailing whitespace.



Create an rk05 disk of our modified sources.

In [49]:
from rt11_utils import create_disk_from_files
create_disk_from_files("tempest_modified/", "tempest_modified_clean.rk05", ["MAC"])

Copy it over to the `atari_build` directory. 

In [50]:
!cp tempest_modified_clean.rk05 ../atari_build/tempest_modified_clean.rk05
!cp tempest_modified_clean.rk05 ../atari_build/tempest_original.rk05

Now you need to go into atari build, and follow the build instructions there. In summary they are:

```
R MAC65
OBJ:ALWELG=ALWELG
OBJ:ALSCO2=ALSCO2
OBJ:ALDIS2=ALDIS2
OBJ:ALEXEC=ALEXEC
OBJ:ALSOUN=ALSOUN
OBJ:ALVROM=ALVROM
^C
```

To run the second batch press `Ctrl-C` to 'close' the assembler first, then paste:

```
R MAC65
OBJ:ALCOIN=ALCOIN
OBJ:ALLANG=ALLANG
OBJ:ALHAR2=ALHAR2
OBJ:ALTES2=ALTES2
OBJ:ALEARO=ALEARO
OBJ:ALVGUT=ALVGUT
^C
```


The command to link our object files is:
```
R LINKM
BIN:ALEXEC/L,ALEXEC/A=OBJ:ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM/C
ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT
^C
```



When we're done building, copy the rk05 disk, which now contains our object files and final binary, back over here.

In [51]:
!cp ../atari_build/tempest_original.rk05 tempest_modified.rk05 

Extract the files from our rk05 disk.

In [52]:
from rt11_utils import extract_files_from_disk
extract_files_from_disk("tempest_modified.rk05", "tempest_mod_object_files")

Compare the `ALEXEC.LDA` we built using our patched sources with the original one from the source dump.

In [62]:
!diff -y -W 150 <(dd if=tempest_modified/ALEXEC.LDA|xxd) <(dd if=tempest_mod_object_files/ALEXEC.LDA|xxd) |grep '  |	'

77+0 records in
77+0 records out
39424 bytes (39 kB, 38 KiB) copied, 5.646e-05 s, 698 MB/s
77+0 records in
77+0 records out
39424 bytes (39 kB, 38 KiB) copied, 5.7418e-05 s, 687 MB/s


Voila! They match!